Encrypted Deep Learning
---
Traing a Deep Learning Model
---
Let's build and train a toy deep learning model

In [0]:
import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F


In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[-1.6],[1.]], requires_grad=True)


In [0]:
encrypted_target = target.fix_precision()
encrypted_target

(Wrapper)>FixedPrecisionTensor>tensor([[                  0],
        [                  0],
        [4611686018427386304],
        [               1000]])

In [0]:
encrypted_target.float_precision()

tensor([[ 0.0000],
        [ 0.0000],
        [-1.6000],
        [ 1.0000]])

In [0]:
# a toy model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).mean()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        

In [0]:
model = Net()


In [0]:
train()

tensor(0.0050)
tensor(0.0040)
tensor(0.0032)
tensor(0.0026)
tensor(0.0020)
tensor(0.0016)
tensor(0.0013)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0005)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(9.4853e-05)
tensor(7.4958e-05)
tensor(5.9525e-05)


In [0]:
predictions = model(data)
predictions

tensor([[0.0025],
        [0.0085],
        [0.9897],
        [0.9980]], grad_fn=<AddmmBackward>)

Encrypted Deep Learning using PySyft
---
Now let's use the library PySyft, which we introduced in the first section of this course, to cary out encrypted learning :


In [0]:
!pip install syft

     |████████████████████████████████| 337kB 4.7MB/s 
     |████████████████████████████████| 399kB 55.6MB/s 
     |████████████████████████████████| 10.1MB 21.3MB/s 
     |████████████████████████████████| 204kB 63.1MB/s 
     |████████████████████████████████| 471kB 61.7MB/s 
     |████████████████████████████████| 773.1MB 21kB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
     |████████████████████████████████| 256kB 74.0MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 122kB 62.8MB/s 
  Created wheel for zstd: filename=zstd-1.4.4.0-cp36-cp36m-linux_x86_64.whl size=1131200 sha256=2bf977b1d96b4bbdb906ec92c5e1d70cfae21df20b1f626636c674587cf62014
  Stored in directory: /root/.cache/pip/wheels/ce/0d/5a/3c2ccf5522d1cd3ecdeb2af8c9889179d7a204376b1d5c042d
Successfully built zstd
  Found existing installation: torch 1.3.1+cu100
    Uninstalling torch-1.3.1+cu100:
      Successfully uninstalled torch-1.3.1+cu100
  Found existing 

In [0]:
import syft as sy
hook = sy.TorchHook(th)

In [0]:
# create a couple of workers
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# encrypt the model and share it among participants
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)
encrypted_prediction

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:6738190149 -> alice:87518776543]
	-> [PointerTensor | me:35845282935 -> bob:63258858520]
	*crypto provider: secure_worker*

In [0]:
results = encrypted_prediction.get()
results

(Wrapper)>FixedPrecisionTensor>tensor([[  3],
        [ 10],
        [987],
        [997]])

In [0]:
results = results.float_precision()
results

tensor([[0.0030],
        [0.0100],
        [0.9870],
        [0.9970]])

Additive Secrete Sharing
---


Additive Secret Sharing is a protocol for Multi-Party Computation. It allows multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x.
Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
x = 5
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted).

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share *= 2 
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. 
Decryption happens by summing all the shares together MODULUS some constant:

In [0]:
x = 5

Q = 23740629843760239486723 # large prime number

bob_x_share = 23552870267 # <- a random number


alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [0]:
(bob_x_share + alice_x_share) % Q

5

 Fixed Precision Encoding
 ---
 Additive secrete sharing works with integers. Thus, to apply this protocol on deep learning, we first need to convert the gradeints to integers. To do so, we used a predifned funciton fixed_precision. In the following, we illustrate its internat implemention in a very simple appraoch:

In [0]:
BASE=10
PRECISION=4

def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q)  / BASE**PRECISION
    

In [0]:
encode(-10)

23740629843760239386723

In [0]:
decode(23740629843760239386723)

-10.0

In [0]:
a = a.torch

---
Apply Addivite Secrete Sharing using PySyft on a real dataset (MNIST or CIFAR10)
---